In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/search-data-1/data.json
/kaggle/input/search-data/questions.json
/kaggle/input/search-data/sentences.json
/kaggle/input/search-index/index_faiss_sbert.index


In [2]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.8 MB/s eta 0:00:0000:0100:01


In [12]:
import json
import faiss

with open('/kaggle/input/search-data-1/data.json','r') as file:
    data=json.load(file)


with open('/kaggle/input/search-data/questions.json','r') as file:
    questions=json.load(file)
    
index=faiss.read_index('/kaggle/input/search-index/index_faiss_sbert.index')


In [16]:
from transformers import pipeline
qa_model=pipeline(task="question-answering",model="deepset/roberta-base-squad2")
#model("Who is the president of the United States?")

In [5]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f77369ffbcd0485e406835cc18b0dcad93e840387d992dbf60c39bd0b5f819ad
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
from sentence_transformers import SentenceTransformer ,util

In [7]:
model=SentenceTransformer('msmarco-distilbert-base-v3',device='cuda')

In [21]:
d=768
def search_func_qa(question,k):
    question_vector=model.encode(question)
    question_vector_norm=question_vector/np.linalg.norm(question_vector)
    Distances,Indexes=index.search(question_vector_norm.reshape(1,d),k)
    #print(*Distances)
    #print(*Indexes)
    docs=[data[a]['text'] for a in Indexes[0]]
    return docs

In [13]:
search_func_qa("What is black death",5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.24880955 0.2449294  0.22833182 0.21622472 0.21317157]
[ 0  2 19 13 15]


['A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'T

In [55]:
a=questions[0]
print(a)
qa_model(question=a,context=''.join(search_func_qa(a,5)))

How many people have died during Black Death?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.6152188777923584,
 'start': 7357,
 'end': 7371,
 'answer': '75–200 million'}

In [63]:
qa_model(question="Whose family name got associated with the Antonine Plague",context=''.join(search_func_qa(a,5)))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'score': 0.6712337732315063,
 'start': 5547,
 'end': 5556,
 'answer': 'Antoninus'}

In [62]:
search_func_qa("Whose family name got associated with the Antonine Plague",5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['The Antonine Plague of 165 to 180 AD, also known as the Plague of Galen (after Galen, the physician who described it), was an ancient pandemic brought to the Roman Empire by troops who were returning from campaigns in the Near East. Scholars have suspected it to have been either smallpox or measles. The plague may have claimed the life of a Roman emperor, Lucius Verus, who died in 169 and was the co-regent of Marcus Aurelius Antoninus, whose family name, Antoninus, has become associated with the pandemic. \nAncient sources agree that the plague appeared first during the Roman siege of the Mesopotamian city Seleucia in the winter of 165–166. Ammianus Marcellinus reported that the plague spread to Gaul and to the legions along the Rhine. Eutropius stated that a large population died throughout the empire. According to the contemporary Roman historian Cassius Dio, the disease broke out again nine years later in 189 AD and caused up to 2,000 deaths a day in Rome, one quarter of those who